Copyright 2025 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
      <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Demos/Emoji-Gemma-on-Web/resources/Convert_Gemma_3_270M_to_LiteRT_for_MediaPipe_LLM_Inference_API.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Convert Gemma 3 270M to LiteRT for use with MediaPipe LLM Inference API

This notebook converts a Gemma 3 270M for use with the [MediaPipe LLM Inference API](https://ai.google.dev/edge/mediapipe/solutions/genai/llm_inference), a library that enables inference on mobile devices or in web browsers. The entire process takes about 15 minutes:

1. Set up the Colab environment
2. Load the model from Hugging Face
3. Convert the model with the AI Edge Torch converter
4. Package the model with the MediaPipe Task bundler
5. Download the model

Gemma 3 270M is designed for task-specific fine-tuning and engineered for efficient performance on mobile, web, and edge devices. You can fine-tune your own model using this [notebook](https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Demos/Emoji-Gemma-on-Web/resources/Fine_tune_Gemma_3_270M_for_emoji_generation.ipynb) and run it in a demo [web app](https://github.com/google-gemini/gemma-cookbook/tree/main/Demos/Emoji-Gemma-on-Web/app-mediapipe) once converted.

## Set up development environment

The first step is to install packages using pip.

In [2]:
%pip uninstall -y tensorflow
%pip install -U tf-nightly==2.21.0.dev20250819 ai-edge-torch==0.6.0 protobuf transformers
%pip install -U jax jaxlib

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.3/532.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.4/439.4 kB 582.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.


Restart the session runtime to ensure you're using the newly installed packages.

## Load the model
To access models on Hugging Face, provide your [Access Token](https://huggingface.co/settings/tokens). You can store it as a Colab secret in the left toolbar by specifying `HF_TOKEN` as the 'Name' and adding your unique token as the 'Value'.

In [3]:
import os
from google.colab import userdata
from huggingface_hub import login
hf_token = ''
login(hf_token)

Specify the Hugging Face repo ID of the model to convert. It'll be saved to your Colab files for conversion.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_author = "li798"                                         #@param {type:"string"}
model_name = "myemoji-gemma-3-270m-it"                    #@param {type:"string"}

repo_id = f"{model_author}/{model_name}"                  # Model to convert
save_path = f"/content/{model_name}"                      # Path to save resized model

model = AutoModelForCausalLM.from_pretrained(repo_id)     # Load the model
tokenizer = AutoTokenizer.from_pretrained(repo_id)        # Load the tokenizer

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.8.0, so it will not be used.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Model and tokenizer saved to /content/myemoji-gemma-3-270m-it


## Convert the model
Convert and quantize the model using the [AI Edge Torch](https://github.com/google-ai-edge/ai-edge-torch) converter. You can adjust the conversions parameters based on your task's requirements:

* `prefill_seq_len`: maximum length of supported input
* `kv_cache_max_len`: maximum of prefill + decode context length
* `quantize`: the quantization scheme. 8-bit integer quantization (INT8) is good for web environments

This takes about 10 minutes. The .tflite model will be saved temporarily to your Colab files.

In [5]:
from ai_edge_torch.generative.examples.gemma3 import gemma3
from ai_edge_torch.generative.utilities import converter
from ai_edge_torch.generative.utilities.export_config import ExportConfig
from ai_edge_torch.generative.layers import kv_cache

# Get model, set export settings, and convert to .tflite
pytorch_model = gemma3.build_model_270m(save_path)
export_config = ExportConfig()
export_config.kvcache_layout = kv_cache.KV_LAYOUT_TRANSPOSED
export_config.mask_as_input = True
converter.convert_to_tflite(
    pytorch_model,
    output_path="/content",
    output_name_prefix=model_name,
    prefill_seq_len=128,
    kv_cache_max_len=512,
    quantize="dynamic_int8",
    export_config=export_config,
)

print (f"Model converted to .tflite and saved to {save_path}")

/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:351: UserWarning: Device capability of jax unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(


Model converted to .tflite and saved to /content/myemoji-gemma-3-270m-it


## Create a MediaPipe Task Bundle

A MediaPipe Task file (.task) bundles the original model tokenizer, the LiteRT model (.tflite), and additional metadata needed to run end-to-end inference with the MediaPipe LLM Inference API.

To use the bundler, install the MediaPipe PyPI package (>0.10.14) in this step as it comes with its own set of dependencies.

In [1]:
%pip install mediapipe

Do a fresh install of `protobuf` and `tensorflow` and restart the Colab runtime to get the latest versions.

In [4]:
%pip uninstall protobuf -y && pip install protobuf==3.20.*
%pip uninstall tensorflow -y -q && pip install tensorflow

Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.20.0 requires protobuf>=5.28.0, but you have protobuf 3.20.3 which is incompatible.
ai-edge-tensorflow 2.21.0.dev20250818 requires protobuf<8.0.0,>=6.31.1, but you have protobuf 3.20.3 which is incompatible.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.3 which is incompatible.
tf-nightly 2.21.0.dev20250819 requires protobuf<8.0.0,>=6.31.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have t

Now, you'll configure and create the Task bundle:

1. Update `tflite_model` to point to the newly converted .tflite model in your Colab files
2. Update `tokenizer_model` to point to the tokenizer.model that was downloaded from Hugging Face Hub.
3. Name your .task file in the `output_filename`.

In [6]:
from mediapipe.tasks.python.genai import bundler

config = bundler.BundleConfig(
    tflite_model="/content/myemoji-gemma-3-270m-it_q8_ekv512.tflite",     # Point to your converted .tflite model
    tokenizer_model="/content/myemoji-gemma-3-270m-it/tokenizer.model",   # Point to the downloaded model's tokenizer.model file
    start_token="<bos>",
    stop_tokens=["<eos>", "<end_of_turn>"],
    output_filename="/content/myemoji-gemma-3-270m-it.task",              # Specify the final model filename
    prompt_prefix="<start_of_turn>user\n",
    prompt_suffix="<end_of_turn>\n<start_of_turn>model\n",
)
bundler.create_bundle(config)

print(f"Model .task bundle saved to {config.output_filename}")

Model .task bundle saved to /content/myemoji-gemma-3-270m-it.task


## Download & run your model on-device

Your model is now ready for on-device inference using the MediaPipe LLM Inference API!

Download the .task file from your Colab environment to use it in your projects.

In [7]:
from google.colab import files

files.download(config.output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Try it in the [emoji generation web app](https://github.com/google-gemini/gemma-cookbook/tree/main/Demos/Emoji-Gemma-on-Web/app-mediapipe) which runs the model directly in the browser. You can also explore [documentation](https://ai.google.dev/edge/mediapipe/solutions/genai/llm_inference) for building cross-platform mobile and web apps.